This example notebook shows how we can train an [image/digit classification](https://pytorch.org/tutorials/beginner/nn_tutorial.html?highlight=mnist)
model based on MNIST dataset, and store it as TileDB array. Firstly, let's import what we need.

In [1]:
import glob
import json
import pickle
import os
import shutil
from pprint import pprint

import tiledb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from tiledb.ml.models.pytorch import PyTorchTileDBModel

TB_KEY = '__TENSORBOARD__'

First let's define the parameters/hyperparameters we will need.

In [2]:
epochs = 1
batch_size_train = 128
learning_rate = 0.01
momentum = 0.5
log_interval = 10

# Set random seeds for anything using random number generation
random_seed = 1

# Disable nondeterministic algorithms
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

We  will also need the DataLoaders API for the dataset. We will also employ TorchVision which let's as load the MNIST
dataset in a handy way. We'll use a batch_size of 64 for training while the values 0.1307 and 0.3081 used for
the Normalize() transformation below are the global mean and standard deviation of the MNIST dataset,
we'll take them as a given here.

In [3]:
data_home = os.path.join(os.path.pardir, 'data')
dataset = torchvision.datasets.MNIST(
    root=data_home, 
    train=True, 
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size_train, shuffle=True)

Moving on, we build our network. We'll use two 2-D convolutional layers followed by two fully-connected
layers. As activation function we'll choose ReLUs and as a means of regularization we'll use two dropout layers.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)


We will now initialise our Neural Network and optimizer.

In [5]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

We continue with the training loop and we iterate over all training data once per epoch. Loading the individual batches
is handled by the DataLoader. We need to set the gradients to zero using optimizer.zero_grad() since PyTorch by default
accumulates gradients. We then produce the output of the network (forward pass) and compute a negative log-likelihodd
loss between the output and the ground truth label. The backward() call we now collect a new set of gradients which we
propagate back into each of the network's parameters using optimizer.step().

In [6]:
train_losses = []
train_counter = []

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    writer.add_scalar("Loss/train", loss, epoch)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

for epoch in range(1, epochs + 1):
  train(epoch)
writer.flush()
writer.close()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.358812
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285138
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.306635
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.270880
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.236739
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.243352
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.183357
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.148380
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.104831
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.004480
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.861931
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.884068
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.797761
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.687489
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.509098
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.766841
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.469026
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.349417
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.291566
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.022800
Train Epoch: 1 [

We can now save the trained model as a TileDB array. In case we want to train  the model further in a later time, we can also save
the optimizer in our TileDB array. In case we will use our model only for inference, we don't have to save the optimizer and we
only keep the model. We first declare a PytTorchTileDB object and initialize it with the corresponding TileDB uri, model and optimizer,
and then save the model as a TileDB array. Finally, we can save any kind of metadata (in any structure, i.e., list, tuple or dictionary)
by passing a dictionary to the meta attribute.

In [7]:
uri = os.path.join(data_home, 'pytorch-mnist-1')
tiledb_model_1 = PyTorchTileDBModel(uri=uri, model=network, optimizer=optimizer)

tiledb_model_1.save(update=False,
                    meta={'epochs': epochs,
                          'train_loss': train_losses},
                    summary_writer=writer)
tiledb_model_1.load_tensorboard()

The above step will create a TileDB array in your working directory. For information about the structure of a dense
TileDB array in terms of files on disk please take a look [here](https://docs.tiledb.com/main/concepts/data-format).
Let's open our TileDB array model and check metadata. Metadata that are of type list, dict or tuple have been JSON
serialized while saving, i.e., we need json.loads to deserialize them.

In [8]:
# Check array directory
pprint(glob.glob(f'{uri}/*'))

# Open in write mode in order to add metadata
model_array_1 = tiledb.open(uri)
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes)  and key != TB_KEY:
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value if key != TB_KEY else "<binary_data>"))

['../data/pytorch-mnist-1/__fragment_meta',
 '../data/pytorch-mnist-1/__meta',
 '../data/pytorch-mnist-1/__fragments',
 '../data/pytorch-mnist-1/__commits',
 '../data/pytorch-mnist-1/__schema']
Key: TILEDB_ML_MODEL_ML_FRAMEWORK, Value: PYTORCH
Key: TILEDB_ML_MODEL_ML_FRAMEWORK_VERSION, Value: 1.12.0.post2
Key: TILEDB_ML_MODEL_PREVIEW, Value: Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Key: TILEDB_ML_MODEL_PYTHON_VERSION, Value: 3.7.13
Key: TILEDB_ML_MODEL_STAGE, Value: STAGING
Key: __TENSORBOARD__, Value: <binary_data>
Key: epochs, Value: 1
Key: train_loss, Value: (2.358812093734741, 2.285137891769409, 2.3066349029541016, 2.2708797454833984, 2.23673939704895, 2.243352174758911, 2.1833572387695312, 2.1483800411224365, 2.1048314571380615, 

As we can see, in array's metadata we have by default information about the backend we used for training (pytorch),
pytorch version, python version and the extra metadata about epochs and training loss that we added.
We can load and check any of the aforementioned without having to load the entire model in memory.
Moreover, we can add any kind of extra information in model's metadata also by opening the TileDB array and adding new keys.

In [9]:
# Open the array in write mode
with tiledb.Array(uri, "w") as A:
    # Keep all history
    A.meta['new_meta'] = json.dumps(['Any kind of info'])

# Check that everything is there
model_array_1 = tiledb.open(uri)
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes) and key != TB_KEY:
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value if key != TB_KEY else "<binary_data>"))

Key: TILEDB_ML_MODEL_ML_FRAMEWORK, Value: PYTORCH
Key: TILEDB_ML_MODEL_ML_FRAMEWORK_VERSION, Value: 1.12.0.post2
Key: TILEDB_ML_MODEL_PREVIEW, Value: Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Key: TILEDB_ML_MODEL_PYTHON_VERSION, Value: 3.7.13
Key: TILEDB_ML_MODEL_STAGE, Value: STAGING
Key: __TENSORBOARD__, Value: <binary_data>
Key: epochs, Value: 1
Key: new_meta, Value: ["Any kind of info"]
Key: train_loss, Value: (2.358812093734741, 2.285137891769409, 2.3066349029541016, 2.2708797454833984, 2.23673939704895, 2.243352174758911, 2.1833572387695312, 2.1483800411224365, 2.1048314571380615, 2.0044803619384766, 1.8619309663772583, 1.8840677738189697, 1.7977608442306519, 1.6874892711639404, 1.5090980529785156, 1.7668406963348389, 1.469025969

For the case of PyTorch models, internally, we save model's state_dict and optimizer's state_dict,
as [variable sized attributes)](https://docs.tiledb.com/main/how-to/arrays/writing-arrays/var-length-attributes)
(pickled), i.e., we can open the TileDB and get only the state_dict of the model or optimizer,
without bringing the whole model in memory. For example, we can load model's and optimizer's state_dict
for model `pytorch-mnist-1` as follows.

In [10]:
# First open arrays
model_array_1 = tiledb.open(uri)[:]

# Load model state_dict
model_1_state_dict = pickle.loads(model_array_1['model_state_dict'].item(0))

# Load optimizer state_dict
optimizer_1_state_dict = pickle.loads(model_array_1['optimizer_state_dict'].item(0))

print(f'Type: {type(model_1_state_dict)} , Keys: {model_1_state_dict.keys()}')
print(f'Type: {type(optimizer_1_state_dict)}, Keys: {optimizer_1_state_dict.keys()}')

Type: <class 'collections.OrderedDict'> , Keys: odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])
Type: <class 'dict'>, Keys: dict_keys(['state', 'param_groups'])


Moving on, we can load the trained models for prediction, evaluation or retraining, as usual with
PyTorch models.

In [11]:
# Place holder for the loaded model
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

# Load returns possible extra attributes, other than model's and optimizer's state dicts. In case there were
# no extra attributes it will return an empty dict
_ = tiledb_model_1.load(model=network, optimizer=optimizer)

What is really nice with saving models as TileDB array, is native versioning based on fragments as described [here](https://docs.tiledb.com/main/concepts/data-format#immutable-fragments). We can load a model, retrain it with new data and update the already existing TileDB model array with new model parameters and metadata. All information, old and new will be there and accessible. This is extremely useful when you retrain with new data or trying different architectures for the same problem, and you want to keep track of all your experiments without having to store different model instances. In our case, let's continue training model_1 with the rest of our dataset and for 2 more epochs. After training is done, you will notice the extra directories and files (fragments) added to `pytorch-mnist-1` TileDB array directory, which keep all versions of the model.

In [12]:
train_losses = []
train_counter = []

# We train for some extra 2 epochs
for epoch in range(1, 2 + 1):
  train(epoch)

# and update
tiledb_model_1 = PyTorchTileDBModel(uri=uri, model=network, optimizer=optimizer)
tiledb_model_1.save(update=True, 
                    meta={'epochs': epochs,
                          'train_loss': train_losses})

# Check array directory
print()
pprint(glob.glob(f'{uri}/*'))

# tiledb.array_fragments() requires TileDB-Py version > 0.8.5
fragments_info = tiledb.array_fragments(uri)

print()
print("====== FRAGMENTS  INFO ======")
print("array uri: {}".format(fragments_info.array_uri))
print("number of fragments: {}".format(len(fragments_info)))

for fragment_num, fragment in enumerate(fragments_info, start=1):
    print()
    print("===== FRAGMENT NUMBER {} =====".format(fragment.num))
    print("fragment uri: {}".format(fragment.uri))
    print("timestamp range: {}".format(fragment.timestamp_range))
    print(
        "number of unconsolidated metadata: {}".format(
            fragment.unconsolidated_metadata_num
        )
    )
    print("version: {}".format(fragment.version))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.573006
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.601681
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.752959
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.664741
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.644972
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.541002
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.667386
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.462301
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.631507
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.613044
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.634575
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.526842
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.575023
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.404579
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.728565
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.524444
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.614918
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.627587
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.631840
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.429085
Train Epoch: 1 [

Finally, a very interesting and useful, for machine learning models, TileDB feature that is described
[here](https://docs.tiledb.com/main/concepts/data-format#groups) and [here](https://docs.tiledb.com/main/how-to/object-management#creating-tiledb-groups)
are groups. Assuming we want to solve the MNIST problem, and we want to try several architectures. We can save each architecture
as a separate TileDB array with native versioning each time it is re-trained, and then organise all models that solve the same problem (MNIST)
as a TileDB array group with any kind of hierarchy. Let's firstly define a new model architecture.

In [13]:
class OtherNet(nn.Module):
    # For the sake of simplicity we just tweak the initial architecture by replacing a relu with relu6.
    def __init__(self):
        super(OtherNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu6(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

Then train it and save it as a new TileDB array.

In [14]:
network = OtherNet()
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

train_losses = []
train_counter = []

for epoch in range(1, epochs + 1):
    train(epoch)

uri2 = os.path.join(data_home, 'pytorch-mnist-2')
tiledb_model_2 = PyTorchTileDBModel(uri=uri2, model=network, optimizer=optimizer)

tiledb_model_2.save(update=False, 
                    meta={'epochs': epochs,
                          'train_loss': train_losses})

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313390
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.527101
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.971071
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.857200
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.513735
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.778414
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.419274
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.507868
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.543029
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.414866
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.563519
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.561157
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.394708
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.480180
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.512548
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.423522
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.407304
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.511762
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.476604
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.387796
Train Epoch: 1 [

Now we can create a TileDB group and organise (in hierarchies, e.g., sophisticated vs less sophisticated) all our
MNIST models as follows.

In [15]:
group = os.path.join(data_home, 'tiledb-pytorch-mnist')
tiledb.group_create(group)
shutil.move(uri, group)
shutil.move(uri2, group)

'../data/tiledb-pytorch-mnist/pytorch-mnist-2'

Any time we can check and query all the available models, including their metadata, for a specific problem like MNIST.

In [16]:
tiledb.ls(group, lambda obj_path, obj_type: print(obj_path, obj_type))

file:///home/gsk/projects/TileDB-ML/examples/data/tiledb-pytorch-mnist/pytorch-mnist-1 array
file:///home/gsk/projects/TileDB-ML/examples/data/tiledb-pytorch-mnist/pytorch-mnist-2 array
